In [25]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY=os.environ['OPENAI_API_KEY']

In [26]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


db = FAISS.load_local("../faiss_index", 
                      OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small"), 
                      allow_dangerous_deserialization=True)

In [27]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [28]:
from langchain_openai import ChatOpenAI

#initialize the LLM we'll use - OpenAI GPT 3.5 Turbo
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-0125")

In [29]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """Given the chat history and a recent user question \
generate a new standalone question \
that can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed or otherwise return it as is."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

retriever_with_history = create_history_aware_retriever(
    llm, retriever, prompt
)

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(retriever_with_history, question_answer_chain)

In [31]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Aetherfloris Ventus?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

print(ai_msg_1["answer"])

Aetherfloris Ventus is a celestial plant with petals lighter than air itself, appearing to float freely, nurtured by the whispers of the winds and clouds. Its stem is nearly invisible but surprisingly strong, leading the petals in a delicate dance with the breeze. The plant's essence, captured in rare vials, is believed to bestow the gift of lightness upon those who partake, freeing the mind and encouraging dreams to take flight.


In [32]:
second_question = "What does a single drop of it do?"

ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})
print(ai_msg_2["answer"])

A single drop of Aetherfloris Ventus is said to lift the spirits, freeing the mind from earthly burdens and encouraging thoughts to soar. It induces a sense of lightness, both in the body and mind, allowing dreams to take flight and inspiring a feeling of weightlessness. The true nature of Aetherfloris Ventus remains a mystery, challenging even dedicated seekers to uncover its secrets.


In [33]:
third_question = "How does it compare to Noctis Umbraherba?"
ai_msg_3 = rag_chain.invoke({"input": third_question, "chat_history": chat_history})
print(ai_msg_3["answer"])

Noctis Umbraherba is a dark, shadowy plant that thrives in the absence of light, with leaves absorbing darkness instead of sunlight. In contrast, Aetherfloris Ventus is a celestial plant with petals lighter than air, appearing to float freely and nurtured by the whispers of the winds and clouds. While Noctis Umbraherba is shrouded in mystery and thrives in darkness, Aetherfloris Ventus embodies the essence of lightness and freedom, reflecting a celestial beauty that captivates with its delicate presence.


In [34]:
fourth_question = "Do you think the Biological section of the Voynich Manuscript is important?"
ai_msg_4 = rag_chain.invoke({"input": fourth_question, "chat_history": chat_history})
print(ai_msg_4["answer"])

Yes, the Biological section of the Voynich Manuscript is significant as it provides intricate anatomical details of mythical beings, possibly reflecting ancient medical knowledge and alchemical practices. The annotated diagrams offer insights into the functions of fantastical organs and systems, possibly used in medicinal or alchemical preparations. These illustrations blend scientific observation with imaginative speculation, inviting further research into how ancient civilizations perceived biology and the body.
